In [ ]:
# 一些参数的设置：学习率、批大小、dropout参数、数据集名称等、训练测试验证比率
import torch
import torch
import torch.nn as nn
import math
import torch.nn.functional as F
class DefaultConfig(object):
    model = 'TRAE_MVPT_MTL'
    use_gpu = True
    if use_gpu:
        device = 'gpu'
    else:
        device = 'cpu'
    load_model_path = None
    # 以上模型基本信息
    data_name = 'iris'
    batch_size = 128
    num_workers = 0
    max_epoch = 500
    patience = patience_acc = patience_mae = 100  # 早停等待轮数
    lr = 0.001
    lr_decay = 0.97
    weight_decay = 1e-5
    train_rate = 0.6
    val_rate = 0.1
    droput = 0.1
    miss_rate = 0.05
    whiten_rate = 0.1 # 白化率
    id_num = 1 # 缺失化的编号（包括划分数据集)
    TOPK = 5 #筛选边的最大数量
    n_hidden = 30 # 隐藏层神经元个数
    use_all_to_train = True # 是否全数据集去训练，用于将缺失值视为变量
    model_save_path_acc = 'zz_saved_model/best_model_acc.pth' #保存最优acc模型状态的位置，暂存用于早停
    model_save_path_mae = 'zz_saved_model/best_model_mae.pth' #保存最优mae模型状态的位置，暂存用于早停
opt = DefaultConfig()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device=", device)
print("opt",opt.num_workers)

In [ ]:
# 常用的函数
import os
import csv
import random
import copy
import numpy as np
import scipy.sparse as sp
import xlwt, xlrd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils import data as torch_data
from sklearn import metrics
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score

def calculate_imputation_metrics(imputed_data, target_data):
    mae = mean_absolute_error(target_data.cpu().numpy(), imputed_data.cpu().numpy())
    rmse = np.sqrt(mean_squared_error(target_data.cpu().numpy(), imputed_data.cpu().numpy()))
    mape = np.mean(np.abs((target_data.cpu().numpy() - imputed_data.cpu().numpy()) / (target_data.cpu().numpy()+1e-8))) * 100
    
    return mae, rmse, mape

# 计算填补损失的MAE, RMSE, MAPE
def calculate_imputation_metrics_mask(imputed_data, target_data, mask): # 有效值mask就行
    # 确保数据和mask都转移到CPU并转换为NumPy数组
    imputed_data_np = imputed_data.cpu().numpy()
    target_data_np = target_data.cpu().numpy()
    mask_np = mask.cpu().numpy()
    
    # 使用mask=0过滤数据
    filtered_imputed_data = imputed_data_np[mask_np == 0]
    filtered_target_data = target_data_np[mask_np == 0]

    # 计算MAE
    mae = mean_absolute_error(filtered_target_data, filtered_imputed_data)
    
    # 计算RMSE
    rmse = np.sqrt(mean_squared_error(filtered_target_data, filtered_imputed_data))
    
    # 计算MAPE，向分母添加一个小的正数以确保不会因为除以零而出错
    epsilon = 1e-8
    mape = np.mean(np.abs((filtered_target_data - filtered_imputed_data) / (filtered_target_data + epsilon))) * 100
    
    return mae, rmse, mape

# 计算分类准确率
def calculate_accuracy(estimated_label, target_labels):
    # estimated_label = torch.argmax(estimated_label, dim=1)
    acc = accuracy_score(target_labels.cpu().numpy(), estimated_label.cpu().numpy())
    
    return acc

# 写入数据行
def writeline_csv(filename, rmse, mae, mape, acc, norm_rmse, norm_mae, norm_mape):
    file_exists = os.path.isfile(filename)
    with open(filename, mode='a', newline='') as file:
        writer = csv.writer(file)
        # 检查文件是否存在，如果不存在，写入头部
        if not file_exists:
            writer.writerow(['Dataset','Miss Rate', 'RMSE', 'MAE', 'MAPE', 'Acc', 'Norm RMSE', 'Norm MAE', 'Norm MAPE'])
        # 写入数据行
        writer.writerow([opt.data_name, opt.miss_rate, rmse, mae, mape, acc, norm_rmse, norm_mae, norm_mape])

def whiten(tensor_,whiten_rate = 0.05):
    tensor = tensor_.clone()
    attr = tensor.shape[1] // 3  # 定义attr的大小
    mask = tensor[:, :attr]  # 提取mask部分

    # 找到mask中所有的有效值（1）的索引
    valid_indices = torch.where(mask == 1) # 这里按照阶给的，所以(valid_indices[0][i],valid_indices[1][i])就是第i个有效值的位置
    # 计算有效值5%的数量
    num_to_whiten = int(whiten_rate * len(valid_indices[0]))
    # 如果需要变为0的数量大于0，则执行变更
    if num_to_whiten > 0:
        # 随机选择一部分有效值的索引来变为0
        indices_to_whiten = torch.randperm(len(valid_indices[0]))[:num_to_whiten]
        # 将这些随机选择的有效值设置为0
        mask[valid_indices[0][indices_to_whiten], valid_indices[1][indices_to_whiten]] = 0

    # 更新原tensor的mask部分
    tensor[:, :attr] = mask
    return tensor

# 定义一个自定义的MSE损失函数，仅计算mask==1的位置的误差
def masked_mse_loss(input_, target, mask):
    # 应用mask，仅保留有效（非缺失）部分
    masked_input = torch.mul(input_, mask)
    masked_target = torch.mul(target, mask)
    
    # 计算有效部分的MSE损失
    loss = torch.sum((masked_input - masked_target) ** 2) / (torch.sum(mask)+1e-8)
    return loss

# 评估函数
def evaluate(opt, model, val_dataloader, feature_label_mask_miss, adj_euclidean, adj_mahalanobis, real_features, norm_real_features, max_zz, min_zz, data_mask_add):
    model.eval()
    predict_all, labels_all = [], []
    RMSE_total, MAE_total, MAPE_total, num_total = 0., 0., 0., 0.
    RMSE_norm, MAE_norm, MAPE_norm = 0., 0., 0.  # 归一化指标初始化
    num_valid = 0 # 有效值的个数，对于多个batch的情况，那就需要累加，而不是一次性计算得到
    with torch.no_grad():
        for batch_evaluate_idx, batch_feature_label_mask_miss in val_dataloader:
            # 获得关键的参数
            attr_dim = real_features.shape[1] #属性维度
            # 根据batch调集子图和子数据输入模型，得到输出
            batch_data_mask_add = data_mask_add[batch_evaluate_idx] #真实的附加mask
            sub_adj_euclidean = adj_euclidean[batch_evaluate_idx][:, batch_evaluate_idx] #真实的子图
            sub_adj_mahalanobis = adj_mahalanobis[batch_evaluate_idx][:, batch_evaluate_idx] #真实的子图
            outputs = model(batch_feature_label_mask_miss[:,:attr_dim], sub_adj_euclidean, sub_adj_mahalanobis, batch_data_mask_add, batch_evaluate_idx)
            # 获得关键的参数
            total_dim = batch_feature_label_mask_miss.shape[1] # 总维度数
            # attr_dim = outputs[0].shape[1] #属性维度
            label_dim = total_dim - 2 * attr_dim #标签维度
            mask = batch_feature_label_mask_miss[:, attr_dim + label_dim:] #输入的掩码矩阵
            norm_input_features = batch_feature_label_mask_miss[:, :attr_dim] * mask #输入的不完整特征，缺失位置为99999->0
            input_features = de_normalize_zz(norm_input_features, max_zz, min_zz) # 输入的不完整特征，其实用不上，但是可以打印看看作为参考
            #！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！
            real_features = features_ori[batch_evaluate_idx] #真实的特征 ！！！！！！！！！本质上是不能用于验证集的，只能用于最终估计！！！！！！！！！！！！！
            norm_real_features = norm_features_ori[batch_evaluate_idx] #归一化后的真实特征
            #############################################################################
            norm_imputed_features = outputs[0].clone() #不完整特征填补结果
            labels_onehot = feature_label_mask_miss[batch_evaluate_idx][:, attr_dim:attr_dim + label_dim] #真实的标签独热版（真实的）
            target_labels = torch.max(labels_onehot, 1)[1] #输入的标签（真实的）（验证集/测试集）
            estimed_labels = outputs[1] # 估计的标签（验证集/测试集）
            # 特征填补和反归一化
            imputed_features = de_normalize_zz(norm_imputed_features, max_zz, min_zz)
            # 计算未归一化指标
            diff = imputed_features - real_features #实际值差距
            mask_indices = np.where(mask.cpu().numpy() == 0) #取缺失位置
            diff_masked = diff[mask_indices] #缺失值位置的填补误差
            real_features_masked = real_features[mask_indices] #用于mape
            
            # 计算归一化指标
            norm_diff = norm_imputed_features - norm_real_features #归一化的值的差距
            norm_diff_masked = norm_diff[mask_indices] #取缺失位置的填补误差
            norm_real_features_masked = norm_real_features[mask_indices] #用于mape
            
            # 计算未归一化指标
            RMSE_total += np.sum(np.square(diff_masked.cpu().numpy())) #实际RMSE
            MAE_total += np.sum(np.abs(diff_masked.cpu().numpy())) 
            MAPE_total += np.sum(np.abs(diff_masked.cpu().numpy() / (1e-9+real_features_masked.cpu().numpy()))) * 100
            # 计算归一化指标 (直接使用outputs和labels进行计算)
            RMSE_norm += np.sum(np.square(norm_diff_masked.cpu().numpy())) #归一化的RMSE
            MAE_norm += np.sum(np.abs(norm_diff_masked.cpu().numpy()))
            MAPE_norm += np.sum(np.abs(norm_diff_masked.cpu().numpy() / (1e-9+norm_real_features_masked.cpu().numpy()))) * 100
            
            # 分类结果的准确率计算
            predictions = torch.max(outputs[1], 1)[1] #估计的标签
            predict_all.extend(predictions.cpu())  # 估计的标签
            labels_all.extend(target_labels.cpu()) # 真实的标签
            num_valid += np.sum(1-mask.cpu().numpy()) #增加本轮缺失值个数，评估的时候是评估的缺失值的填补效果
    # 计算平均指标
    rmse = np.sqrt(RMSE_total / (num_valid+1e-8))
    mae = MAE_total / (num_valid+1e-8)
    mape = MAPE_total / (num_valid+1e-8)
    rmse_norm = np.sqrt(RMSE_norm / (num_valid+1e-8))
    mae_norm = MAE_norm / (num_valid+1e-8)
    mape_norm = MAPE_norm / (num_valid+1e-8)
    acc = metrics.accuracy_score(labels_all, predict_all) if labels_all else 0

    return rmse, mae, mape, rmse_norm, mae_norm, mape_norm, acc


In [ ]:
# 模型架构
import torch
import torch.nn as nn
import math
import torch.nn.functional as F
import pandas as pd

class BasicModule(torch.nn.Module):
    def __init__(self):
        super(BasicModule, self).__init__()
        self.model_name = str(type(self))

    def load(self, path):
        self.load_state_dict(torch.load(path))

    def save(self, name=None):
        if name is None:
            prefix = './checkpoints/' + self.model_name
        torch.save(self.state_dict(), prefix)
        return name

class TRAE_MVPT_MTL(BasicModule):
    def __init__(self, hidden_dim, num_classes, global_mask_shape, data_class, model_type):
        super(TRAE_MVPT_MTL, self).__init__()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        input_feature_dim = global_mask_shape[1]
        self.model_name = 'TRAE_MVPT_MTL' # 模型名
        self.dropout_layer = nn.Dropout(p=opt.droput) # 随机无效化层 
        self.relu = nn.ReLU() # 激活函数
        self.graph_sage_layer1 = Graphsage(input_feature_dim, hidden_dim) # 输入-隐藏
        self.graph_sage_layer2 = Graphsage_hidden(hidden_dim, hidden_dim) # 隐藏（+2*输入）-隐藏
        self.feature_transform = nn.Linear(hidden_dim, input_feature_dim) # 隐藏转化为特征输出
        self.class_transform = nn.Linear(hidden_dim, num_classes) # 隐藏转化为分类输出
        # 初始化全局缺失值参数矩阵
        self.global_missing_values = nn.Parameter(torch.rand(global_mask_shape))
        self.data_class = data_class
    def forward(self, norm_missing_data, batch_idx):
        # 获取mask
        batch_mask = ~torch.isnan(norm_missing_data).to(self.device)
        # 使用batch_indices从全局缺失值参数矩阵中提取当前batch的缺失值参数
        batch_missing_values = self.global_missing_values[batch_idx]
        # 将缺失值替换为缺失值变量的当前值
        input_data = torch.where(batch_mask.bool(), norm_missing_data, batch_missing_values)
        # 输入层-隐藏层(输入属性去跟踪处理，融入图信息)
        hidden_states_1 = self.graph_sage_layer1(input_data)
        # # 隐藏层-隐藏层（隐藏状态的中间过渡，不融入图信息）
        hidden_states_2 = self.graph_sage_layer2(hidden_states_1)
        #用于存放填补结果
        concatenated_result = torch.tensor([], device=self.device)
        # 线性层-填补输出，最后一个隐藏状态是不去跟踪的，用于分类
        for j, hidden in enumerate(hidden_states_2[:-1]):
            transformed_data_j = self.feature_transform(hidden)
            feature_j = transformed_data_j[:, j].unsqueeze(1)  # 使用unsqueeze保持维度一致性，以便于拼接
            concatenated_result = torch.cat((concatenated_result, feature_j), dim=1) # 将这个特征与之前的结果进行拼接
        # 线性层-分类输出
        class_logits = self.class_transform(hidden_states_2[-1])
        # 分类结果经过激活函数，强化锐度，填补因为是经过了Z归一化的，所以不用
        imputed_data = concatenated_result
        estimated_label = F.log_softmax(class_logits, dim=1)

        return input_data, imputed_data, estimated_label

class Graphsage(nn.Module):  # 聚合信息的层
    def __init__(self, input_feature_dim, output_feature_dim):
        super(Graphsage, self).__init__()
        self.in_features = input_feature_dim
        self.model_name = 'Graphsage'
        self.W = nn.Parameter(torch.zeros(size=(1 * input_feature_dim, output_feature_dim)))
        self.bias = nn.Parameter(torch.zeros(output_feature_dim))
        self.reset_parameters()
        self.bn = nn.BatchNorm1d(output_feature_dim)  # 添加BatchNorm层
        
    def reset_parameters(self):  # 初始化参数
        stdv = 1. / (math.sqrt(self.W.size(1)) + 1e-8)
        self.W.data.uniform_(-stdv, stdv)
        self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input_data):
        hidden_states = []  # 用于存储每个属性对应的隐藏状态
        n_attributes = input_data.size(1)  # 输入属性的数量
        
        # 拼接特征和聚合信息
        concatenated_features = input_data
        for j in range(n_attributes): # 去跟踪的实现
            # 创建一个新的concatenated_features副本以便修改
            modified_features = concatenated_features.clone()
            # 将属性j对应的输入concatenated_features中对应的位置的值置为0
            modified_features[:, j] = 0
            # # 将属性j对应的aggregated_data中对应的位置的值也置为0
            # modified_features[:, n_attributes + j] = 0

            # 计算transformed_features
            transformed_features = torch.mm(modified_features, self.W) + self.bias
            # transformed_features = self.bn(transformed_features)  # 应用BatchNorm
            
            # 将计算得到的transformed_features加入列表
            hidden_states.append(transformed_features)

        # 用于分类的没有去跟踪的隐藏状态
        hidden_states.append(torch.mm(concatenated_features, self.W) + self.bias)
        
        # 返回所有隐藏状态的列表
        return hidden_states

class Graphsage_hidden(nn.Module): #聚合隐藏信息的层
    def __init__(self, hidden_feature_dim, output_feature_dim):
        super(Graphsage_hidden, self).__init__()
        self.model_name = 'Graphsage_hidden'
        self.W = nn.Parameter(torch.zeros(size=(hidden_feature_dim, output_feature_dim)))
        self.bias = nn.Parameter(torch.zeros(output_feature_dim))
        self.reset_parameters()
        self.bn = nn.BatchNorm1d(output_feature_dim)  # 添加BatchNorm层

    def reset_parameters(self): # 初始化参数
        # Initialize parameters uniformly based on layer dimensions
        stdv = 1. / math.sqrt(self.W.size(1))
        self.W.data.uniform_(-stdv, stdv)
        self.bias.data.uniform_(-stdv, stdv)

    def forward(self, hidden_states):
        new_hidden_states = []
        # 计算下一个隐藏状态列表（注意，前s个是去跟踪了的，最后一个是不去跟踪的用于分类的）
        for hidden in hidden_states:
            transformed_hidden = torch.mm(hidden, self.W) + self.bias #做一个映射，返回原本的属性维度
            new_hidden_states.append(transformed_hidden)
        return new_hidden_states

In [ ]:
# 数据类
from torch.utils import data as torch_data
import numpy as np

class Dataload_zz(torch_data.Dataset):
    def __init__(self, norm_missing_data, norm_full_data, labels_onehot):
        self.norm_missing_data = norm_missing_data #归一化的缺失数据样本
        self.norm_full_data = norm_full_data #归一化的完整数据样本
        self.labels_onehot = labels_onehot # 标签

    def __len__(self):
        # 数据集的总长度等于任一数据的长度
        return len(self.norm_missing_data)

    def __getitem__(self, idx):
        batch_norm_missing_data_sample = self.norm_missing_data[idx]
        batch_norm_full_data_sample = self.norm_full_data[idx]
        labels_onehot = self.labels_onehot[idx]
        return idx, batch_norm_missing_data_sample, batch_norm_full_data_sample, labels_onehot


class Dataset_zz:
    def __init__(self, opt):
        self.data_name = opt.data_name
        self.miss_rate = int(100 * opt.miss_rate)
        self.id_num = opt.id_num
        self.TOPK = opt.TOPK
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.load_data()
        self.normalize_data()
        self.encode_labels()
        self.generate_masks()
        # self.generate_adjacency_matrices() # TRAE_MVPT_MTL不使用邻接矩阵
        self.define_indices() # 定义索引
        self.to_tensor() # 转device上的tensor
    def load_data(self):
        base_path = f'datasets/{self.data_name}/{self.data_name}'
        self.missing_data_train = np.array(pd.read_csv(f'{base_path}_train_RANDOM_{self.miss_rate}%_NUM_{self.id_num}.csv', header=None))[:, :-1]
        self.missing_data_val = np.array(pd.read_csv(f'{base_path}_val_RANDOM_{self.miss_rate}%_NUM_{self.id_num}.csv', header=None))[:, :-1]
        self.missing_data_test = np.array(pd.read_csv(f'{base_path}_test_RANDOM_{self.miss_rate}%_NUM_{self.id_num}.csv', header=None))[:, :-1]
        self.missing_data_all = np.concatenate((self.missing_data_train, self.missing_data_val, self.missing_data_test), axis=0)
        
        self.train_set = np.array(pd.read_csv(f'{base_path}_train_REAL_{self.miss_rate}%_NUM_{self.id_num}.csv', header=None))
        self.val_set = np.array(pd.read_csv(f'{base_path}_val_REAL_{self.miss_rate}%_NUM_{self.id_num}.csv', header=None))
        self.test_set = np.array(pd.read_csv(f'{base_path}_test_REAL_{self.miss_rate}%_NUM_{self.id_num}.csv', header=None))
        self.all_set = np.concatenate((self.train_set, self.val_set, self.test_set), axis=0)

        self.train_data, self.train_label = self.train_set[:, :-1], self.train_set[:, -1]
        self.val_data, self.val_label = self.val_set[:, :-1], self.val_set[:, -1]
        self.test_data, self.test_label = self.test_set[:, :-1], self.test_set[:, -1]
        self.all_data, self.all_label = self.all_set[:, :-1], self.all_set[:, -1]

    def normalize_data(self):
        _, self.all_mean, self.all_std = self.normalization(self.missing_data_all.copy())
        self.missing_data_train_norm = self.normalization_with_mean_std(self.missing_data_train.copy(), self.all_mean, self.all_std)
        self.missing_data_val_norm = self.normalization_with_mean_std(self.missing_data_val.copy(), self.all_mean, self.all_std)
        self.missing_data_test_norm = self.normalization_with_mean_std(self.missing_data_test.copy(), self.all_mean, self.all_std)
        self.missing_data_all_norm = self.normalization_with_mean_std(self.missing_data_all.copy(), self.all_mean, self.all_std)
        self.train_data_norm = self.normalization_with_mean_std(self.train_data.copy(), self.all_mean, self.all_std)
        self.val_data_norm = self.normalization_with_mean_std(self.val_data.copy(), self.all_mean, self.all_std)
        self.test_data_norm = self.normalization_with_mean_std(self.test_data.copy(), self.all_mean, self.all_std)
        self.all_data_norm = self.normalization_with_mean_std(self.all_data.copy(), self.all_mean, self.all_std)
        
    # 数据Z-SCORE标准化
    @staticmethod
    def normalization(data): 
        temp = np.array(data)
        mean = np.nanmean(temp, axis=0) # 计算均值时忽略nan
        std = np.nanstd(temp, axis=0) # 计算标准差时忽略nan
        temp_masked = np.ma.masked_invalid(temp) # 创建掩码数组，将nan替换为掩码值
        temp = (temp_masked - mean) / (std + 1e-8) # 增加了一个小常数防止除以0
        temp = temp.filled(np.nan) # 将掩码值替换为nan
        return temp, mean, std

    # 使用已知的均值和标准差进行Z-SCORE标准化
    @staticmethod
    def normalization_with_mean_std(data, mean, std): 
        temp = np.array(data)
        temp_masked = np.ma.masked_invalid(temp) # 创建掩码数组，将nan替换为掩码值
        temp = (temp_masked - mean).astype(np.float32) / (std + 1e-8).astype(np.float32) # 增加了一个小常数
        temp = temp.filled(np.nan) # 将掩码值替换为nan
        return temp

    # 反Z-SCORE标准化
    @staticmethod
    def denormalization_with_mean_std(data, mean, std):
        return data * (std + 1e-8) + mean
    
    def encode_labels(self):
        # 步骤 1: 确定所有数据集中所有唯一类别的并集
        unique_all_labels = np.unique(np.concatenate((self.train_label, self.val_label, self.test_label)))
        # 步骤 2: 基于所有唯一类别创建独热编码映射
        self.classes_dict = {label: np.eye(len(unique_all_labels))[i, :] for i, label in enumerate(unique_all_labels)}
        # 对所有数据集应用独热编码
        self.train_label_onehot = self.encode_onehot(self.train_label)
        self.val_label_onehot = self.encode_onehot(self.val_label)
        self.test_label_onehot = self.encode_onehot(self.test_label)
        self.all_label_onehot = self.encode_onehot(self.all_label)

    def encode_onehot(self, labels):
        # 使用 np.vectorize 以向量化的方式应用映射，提高效率
        map_func = np.vectorize(self.classes_dict.get, otypes=[np.ndarray])
        labels_onehot = np.array(list(map_func(labels)), dtype=np.int32)
        return labels_onehot
    
    def generate_masks(self):
        # 为训练、验证和测试集的缺失数据生成掩码矩阵
        self.mask_train = (~np.isnan(self.missing_data_train)).astype(int)
        self.mask_val = (~np.isnan(self.missing_data_val)).astype(int)
        self.mask_test = (~np.isnan(self.missing_data_test)).astype(int)
        self.mask_all = (~np.isnan(self.missing_data_all)).astype(int)
    
    def generate_adjacency_matrices(self):
        self.adj_train = self.get_adj_euclidean(self.missing_data_train, self.mask_train)
        self.adj_val = self.get_adj_euclidean(self.missing_data_val, self.mask_val)
        self.adj_test = self.get_adj_euclidean(self.missing_data_test, self.mask_test)
        self.adj_all = self.get_adj_euclidean(self.missing_data_all, self.mask_all)
        
    # 获取数据的每个属性的方差
    def get_data_var(self, data, data_mask):
        (n_sample, n_attribute) = data.shape
        data_var = np.zeros(n_attribute)
        # 遍历每个属性
        for i in range(n_attribute):
            valid_data = data[data_mask[:, i] == 1, i]  # 仅选择该属性中有效的数据
            if valid_data.size > 0:  # 确保有有效数据
                data_mean = np.mean(valid_data)  # 计算有效数据的均值
                # 计算有效数据的方差
                var = np.sum((valid_data - data_mean) ** 2) / valid_data.size
                data_var[i] = var  # 存储每个属性的方差
            else:
                data_var[i] = np.nan  # 如果没有有效数据，则标记为NaN
        return data_var

    # 计算两个样本之间的距离
    def get_dis_euclidean(self, x, y, data_var, x_mask, y_mask):
        # 获取同时在x和y中有效的位置
        valid_mask = x_mask * y_mask
        valid_indices = np.where(valid_mask == 1)[0]
        # 仅选取有效位置的值进行距离计算
        if valid_indices.size > 0:
            x_valid = x[valid_indices]
            y_valid = y[valid_indices]
            data_var_valid = data_var[valid_indices]
            n_attributes = x.shape[0]  # 总属性数
            # 计算有效位置上的距离平方和, 还需要除以属性的方差使得每个属性的总差异（用方差/标准差衡量的差异，都是1）的重要性等同
            distance_squared_sum = (((x_valid - y_valid) ** 2)/ data_var_valid).sum()
            # 缩放距离平方和：将距离平方和除以有效值个数，乘以属性总数
            scaled_distance_squared = distance_squared_sum * n_attributes / valid_indices.size
            # 计算缩放后的欧式距离
            dis = np.sqrt(scaled_distance_squared)
        else:
            # 如果没有有效的共同位置，则距离设为无穷大或其他标记值
            dis = np.inf
        return dis

    # 构建邻接矩阵
    def get_adj_euclidean(self, data, matrix_mask):
        n_sample = data.shape[0]
        matrix_adj = np.zeros((n_sample, n_sample))
        data_var = self.get_data_var(data, matrix_mask)
        for i in range(n_sample):
            for j in range(i + 1, n_sample):
                Dij = self.get_dis_euclidean(data[i], data[j], data_var, matrix_mask[i], matrix_mask[j])
                if Dij != 0 and Dij != np.inf: 
                    matrix_adj[i][j] = matrix_adj[j][i] = 1 / Dij
        for i in range(n_sample):
            row = matrix_adj[i]
            if np.count_nonzero(row) > self.TOPK:
                threshold = np.partition(row, -self.TOPK)[-self.TOPK]
                matrix_adj[i][matrix_adj[i] < threshold] = 0
        row_sums = matrix_adj.sum(axis=1) + 1e-6
        matrix_adj = matrix_adj / row_sums[:, np.newaxis]
        return matrix_adj
    
    def define_indices(self):
        # 计算训练、验证、测试数据的长度
        train_len = len(self.train_data)
        val_len = len(self.val_data)
        test_len = len(self.test_data)

        # 根据数据长度计算索引范围
        self.train_indices = range(0, train_len)
        self.val_indices = range(train_len, train_len + val_len)
        self.test_indices = range(train_len + val_len, train_len + val_len + test_len)

        # 也可以将索引转换为list，便于后续操作
        self.train_indices = list(self.train_indices)
        self.val_indices = list(self.val_indices)
        self.test_indices = list(self.test_indices)
        
    def to_tensor(self): #将所有数据转换为tensor，并移动到指定的device上
        self.missing_data_train = torch.from_numpy(self.missing_data_train).float().to(self.device)
        self.missing_data_val = torch.from_numpy(self.missing_data_val).float().to(self.device)
        self.missing_data_test = torch.from_numpy(self.missing_data_test).float().to(self.device)
        self.missing_data_all = torch.from_numpy(self.missing_data_all).float().to(self.device)

        self.missing_data_train_norm = torch.from_numpy(self.missing_data_train_norm).float().to(self.device)
        self.missing_data_val_norm = torch.from_numpy(self.missing_data_val_norm).float().to(self.device)
        self.missing_data_test_norm = torch.from_numpy(self.missing_data_test_norm).float().to(self.device)
        self.missing_data_all_norm = torch.from_numpy(self.missing_data_all_norm).float().to(self.device)

        self.train_data =  torch.from_numpy(self.train_data).float().to(self.device)
        self.val_data =  torch.from_numpy(self.val_data).float().to(self.device)
        self.test_data =  torch.from_numpy(self.test_data).float().to(self.device)
        self.all_data = torch.from_numpy(self.all_data).float().to(self.device)

        self.train_data_norm =  torch.from_numpy(self.train_data_norm).float().to(self.device)
        self.val_data_norm =  torch.from_numpy(self.val_data_norm).float().to(self.device)
        self.test_data_norm =  torch.from_numpy(self.test_data_norm).float().to(self.device)
        self.all_data_norm = torch.from_numpy(self.all_data_norm).float().to(self.device)

        self.train_label = torch.from_numpy(self.train_label).float().to(self.device)
        self.val_label = torch.from_numpy(self.val_label).float().to(self.device)
        self.test_label = torch.from_numpy(self.test_label).float().to(self.device)
        self.all_label = torch.from_numpy(self.all_label).float().to(self.device)

        self.train_label_onehot = torch.from_numpy(self.train_label_onehot).float().to(self.device)
        self.val_label_onehot = torch.from_numpy(self.val_label_onehot).float().to(self.device)
        self.test_label_onehot = torch.from_numpy(self.test_label_onehot).float().to(self.device)
        self.all_label_onehot = torch.from_numpy(self.all_label_onehot).float().to(self.device)

        self.mask_train = torch.from_numpy(self.mask_train).float().to(self.device)
        self.mask_val = torch.from_numpy(self.mask_val).float().to(self.device)
        self.mask_test = torch.from_numpy(self.mask_test).float().to(self.device)
        self.mask_all = torch.from_numpy(self.mask_all).float().to(self.device)
        
        # self.adj_train = torch.from_numpy(self.adj_train).float().to(self.device)
        # self.adj_val = torch.from_numpy(self.adj_val).float().to(self.device)
        # self.adj_test = torch.from_numpy(self.adj_test).float().to(self.device)
        # self.adj_all = torch.from_numpy(self.adj_all).float().to(self.device)

        self.all_mean = torch.from_numpy(self.all_mean).float().to(self.device)
        self.all_std = torch.from_numpy(self.all_std).float().to(self.device)

        self.train_indices = torch.tensor(self.train_indices)
        self.val_indices = torch.tensor(self.val_indices)
        self.test_indices = torch.tensor(self.test_indices)


In [ ]:
import matplotlib.pyplot as plt
# 运行代码的主函数-这个是将缺失值视为变量的版本
opt.model = 'TRAE_MVPT_MTL'
# 遍历数据集名称
for opt.data_name in ['wdbc']: #'iris', 'autompg', 'banknote', 'glass', 'seeds', 'pima', 'texture'
    # 遍历不同的数据编号
    for opt.id_num in [5,4,3,2]: # 1,2,3,4,5
        # 遍历不同的缺失率
        for opt.miss_rate in [0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7]: #0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7
            # 加载数据并进行预处理
            DATA_ZZ = Dataset_zz(opt)
            # 准备训练、验证和测试数据, 这里的是个人的数据导入
            all_data_zz = Dataload_zz(DATA_ZZ.missing_data_all_norm, DATA_ZZ.all_data_norm, DATA_ZZ.all_label_onehot) 
            # train_data_zz = Dataload_zz(DATA_ZZ.missing_data_train_norm, DATA_ZZ.train_data_norm, DATA_ZZ.train_label_onehot, DATA_ZZ.adj_train) 
            # val_data_zz = Dataload_zz(DATA_ZZ.missing_data_val_norm, DATA_ZZ.val_data_norm, DATA_ZZ.val_label_onehot, DATA_ZZ.adj_val)
            # test_data_zz = Dataload_zz(DATA_ZZ.missing_data_test_norm, DATA_ZZ.test_data_norm, DATA_ZZ.test_label_onehot, DATA_ZZ.adj_test)
            global_train_idx, global_val_idx, global_test_idx = DATA_ZZ.train_indices, DATA_ZZ.val_indices, DATA_ZZ.test_indices

            # 初始化模型
            n_class = DATA_ZZ.train_label_onehot.shape[1] #不重复的类别数
            global_mask_shape = DATA_ZZ.all_data.shape #训练集数据集/掩码矩阵形状，用于将缺失值视为变量动态优化
            model = TRAE_MVPT_MTL(opt.n_hidden, n_class, global_mask_shape, DATA_ZZ, 'all').to(device) #构建模型
            
            # 加载预训练模型，如果指定了模型路径
            if opt.load_model_path:
                model.load(opt.load_model_path)
            
            # 准备数据加载器
            all_dataloader = DataLoader(all_data_zz, opt.batch_size, shuffle=True, num_workers=opt.num_workers)
            # train_dataloader = DataLoader(train_data_zz, opt.batch_size, shuffle=True, num_workers=opt.num_workers)
            # val_dataloader = DataLoader(val_data_zz, opt.batch_size, shuffle=True, num_workers=opt.num_workers)
            # test_dataloader = DataLoader(test_data_zz, opt.batch_size, shuffle=True, num_workers=opt.num_workers)
            
            # 定义损失函数和优化器
            criterion_classicfic = torch.nn.NLLLoss().to(device)
            criterion_imputation = torch.nn.MSELoss().to(device)
            optimizer = torch.optim.Adam(model.parameters(), lr=opt.lr, weight_decay=opt.weight_decay)
            
            # # 初始化早停相关参数
            # patience_acc, patience_mae = opt.patience_acc, opt.patience_mae  # 早停等待轮数
            # best_acc = 0.0  # 目前为止最好的准确率
            # best_mae = float('inf')  # 目前为止最低的MAE
            # wait_acc = wait_mae = 0  # 早停等待计数器
            # early_stop_triggered_acc = early_stop_triggered_mae = False # 标记是否早停
            # test_performed_acc = test_performed_mae = False  # 标记是否已经执行了基于ACC和MAE的测试

            # 初始化存储指标的列表
            train_losses, val_losses, test_losses = [], [], []
            train_maes, train_rmses, train_mapes, train_accs = [], [], [], []
            val_maes, val_rmses, val_mapes, val_accs = [], [], [], []
            test_maes, test_rmses, test_mapes, test_accs = [], [], [], []
            train_mae_accs, val_mae_accs, test_mae_accs = [], [], []
            train_de_maes, train_de_rmses, train_de_mapes = [], [], []
            val_de_maes, val_de_rmses, val_de_mapes = [], [], []
            test_de_maes, test_de_rmses, test_de_mapes = [], [], []


            best_val_loss = float('inf')
            patience, triggered = opt.patience, 0  # 早停的耐心值和触发计数
            # 开始训练模型(验证和测试都在其中获取idx)
            for epoch in range(opt.max_epoch):
                # 重置每个epoch开始时的累积变量
                epoch_input_data_list = []
                epoch_imputed_data_list = []
                epoch_target_data_list = [] #验证的时候是用真实值而不是缺失值变量的重构值
                epoch_real_data_list = [] # 真实值
                epoch_mask_list = [] #掩码矩阵
                epoch_estimated_label_onehot_list = []
                epoch_target_label_onehot_list = []
                epoch_all_idx_list = []

                for batch_data in all_dataloader: #按照批次取遍序号（全）
                    # 解包数据
                    batch_all_idx, batch_norm_missing_data, batch_norm_full_data, batch_label_onehot = batch_data
                    batch_mask = ~torch.isnan(batch_norm_missing_data).to(device) # 将True/False反转，因为~操作符在bool张量上不工作
                    
                    # 输入模型前向传播，获取输出变量
                    outputs = model(batch_norm_missing_data, batch_all_idx) #模型输出结果
                    input_data, imputed_data, estimated_label_onehot = outputs # 解包模型输出
                    target_data = batch_norm_full_data * batch_mask + input_data * (~batch_mask) # 目标训练结果是完整值的重构+缺失值变量的重构
                    target_label_onehot = batch_label_onehot # 目标标签就是原始标签
                    target_labels = torch.argmax(target_label_onehot, dim=1) # 获取分类标签

                    # 提取出训练idx，验证idx，测试idx
                    # 将全局索引转换为一个布尔掩码，用于标记批次中每个样本是否属于对应的集合
                    mask_train_idx = torch.isin(batch_all_idx, global_train_idx)
                    mask_val_idx = torch.isin(batch_all_idx, global_val_idx)
                    mask_test_idx = torch.isin(batch_all_idx, global_test_idx)
                    # 使用布尔掩码来选择当前批次中属于训练、验证和测试集的样本索引
                    batch_train_idx = batch_all_idx[mask_train_idx]
                    batch_val_idx = batch_all_idx[mask_val_idx]
                    batch_test_idx = batch_all_idx[mask_test_idx]
                    # 使用 torch.where 和布尔掩码获取在batch_all_idx中的相对位置
                    batch_train_relative_idx = torch.where(mask_train_idx)
                    batch_val_relative_idx = torch.where(mask_val_idx)
                    batch_test_relative_idx = torch.where(mask_test_idx)

                    # 填补损失，仅完整值的重构误差
                    loss_imputation = criterion_imputation(imputed_data * batch_mask, target_data * batch_mask) # 重构误差+缺失值的变化误差@全数据集的@
                    loss_imputation = loss_imputation/(1-opt.miss_rate)
                    # 分类损失，仅训练集的
                    loss_classicfic_train = criterion_classicfic(estimated_label_onehot[batch_train_relative_idx], target_labels[batch_train_relative_idx]) #分类误差使用训练集的分类误差
                    # loss_classicfic_val = criterion_classicfic(estimated_label[batch_val_relative_idx], target_labels[batch_val_relative_idx]) #分类误差使用训练集的分类误差
                    # loss_classicfic_test = criterion_classicfic(estimated_label[batch_test_relative_idx], target_labels[batch_test_relative_idx]) #分类误差使用训练集的分类误差
                    # 总训练损失
                    batch_model_loss_train = loss_classicfic_train + loss_imputation # 填补误差+分类误差
                    # batch_model_loss_val = loss_classicfic_val + loss_imputation # 填补误差和分类误差各一半
                    # batch_model_loss_test = loss_classicfic_test + loss_imputation # 填补误差和分类误差各一半
                    
                    # 优化
                    optimizer.zero_grad()
                    batch_model_loss_train.backward(retain_graph=True)
                    optimizer.step()

                    # 在这里累积当前批次的相关变量
                    epoch_input_data_list.append(input_data.detach())
                    epoch_imputed_data_list.append(imputed_data.detach())
                    epoch_real_data_list.append(batch_norm_full_data.detach()) # 真实值
                    epoch_mask_list.append(batch_mask.detach()) #掩码矩阵
                    epoch_target_data_list.append(target_data.detach())
                    epoch_estimated_label_onehot_list.append(estimated_label_onehot.detach())
                    epoch_target_label_onehot_list.append(target_label_onehot.detach())
                    epoch_all_idx_list.append(batch_all_idx.detach())
                ###################################################
                #               验证和测试                        #
                ###################################################
                # 在epoch级别上累积数据
                # （1）归一化填补结果，输入，目标值
                epoch_input_data = torch.cat(epoch_input_data_list)
                epoch_imputed_data = torch.cat(epoch_imputed_data_list)
                epoch_target_data = torch.cat(epoch_target_data_list) #缺失变量的输入+有效值
                epoch_real_data = torch.cat(epoch_real_data_list) # 真实值
                epoch_mask = torch.cat(epoch_mask_list) #掩码矩阵
                # （2）反归一化填补结果，输入，目标值
                epoch_de_input_data = DATA_ZZ.denormalization_with_mean_std(epoch_input_data.clone(), DATA_ZZ.all_mean, DATA_ZZ.all_std)
                epoch_de_imputed_data = DATA_ZZ.denormalization_with_mean_std(epoch_imputed_data.clone(), DATA_ZZ.all_mean, DATA_ZZ.all_std)
                epoch_de_target_data = DATA_ZZ.denormalization_with_mean_std(epoch_target_data.clone(), DATA_ZZ.all_mean, DATA_ZZ.all_std) #缺失变量的输入+有效值
                epoch_de_real_data = DATA_ZZ.denormalization_with_mean_std(epoch_real_data.clone(), DATA_ZZ.all_mean, DATA_ZZ.all_std) # 真实值
                # （3）分类结果
                epoch_estimated_label_onehot = torch.cat(epoch_estimated_label_onehot_list)
                epoch_target_label_onehot = torch.cat(epoch_target_label_onehot_list)
                epoch_all_idx = torch.cat(epoch_all_idx_list)
                epoch_estimated_labels = torch.argmax(epoch_estimated_label_onehot, dim=1) # 获取分类标签
                epoch_target_labels = torch.argmax(epoch_target_label_onehot, dim=1) # 获取分类标签

                # 提取出训练idx，验证idx，测试idx
                epoch_mask_train_idx = torch.isin(epoch_all_idx, global_train_idx)
                epoch_mask_val_idx = torch.isin(epoch_all_idx, global_val_idx)
                epoch_mask_test_idx = torch.isin(epoch_all_idx, global_test_idx)
                # 使用布尔掩码来选择当前批次中属于训练、验证和测试集的样本索引
                epoch_batch_train_idx = epoch_all_idx[epoch_mask_train_idx]
                epoch_batch_val_idx = epoch_all_idx[epoch_mask_val_idx]
                epoch_batch_test_idx = epoch_all_idx[epoch_mask_test_idx]
                # 使用 torch.where 和布尔掩码获取在batch_all_idx中的相对位置
                epoch_train_relative_idx = torch.where(epoch_mask_train_idx)
                epoch_val_relative_idx = torch.where(epoch_mask_val_idx)
                epoch_test_relative_idx = torch.where(epoch_mask_test_idx)

                # 填补损失，仅完整值的重构误差
                epoch_loss_imputation = criterion_imputation(epoch_input_data * epoch_mask, epoch_imputed_data * epoch_mask) # 重构误差+缺失值的变化误差@全数据集的@
                epoch_loss_imputation = epoch_loss_imputation/(1-opt.miss_rate)
                # 分类损失，仅训练集的
                epoch_loss_classicfic_train = criterion_classicfic(epoch_estimated_label_onehot[epoch_train_relative_idx], epoch_target_labels[epoch_train_relative_idx]) #分类误差使用训练集的分类误差
                epoch_loss_classicfic_val = criterion_classicfic(epoch_estimated_label_onehot[epoch_val_relative_idx], epoch_target_labels[epoch_val_relative_idx]) #分类误差使用训练集的分类误差
                epoch_loss_classicfic_test = criterion_classicfic(epoch_estimated_label_onehot[epoch_test_relative_idx], epoch_target_labels[epoch_test_relative_idx]) #分类误差使用训练集的分类误差
                # 总损失
                epoch_model_loss_train = epoch_loss_classicfic_train + epoch_loss_imputation # 填补误差+分类误差
                epoch_model_loss_val = epoch_loss_classicfic_val + epoch_loss_imputation # 填补误差和分类误差各一半
                epoch_model_loss_test = epoch_loss_classicfic_test + epoch_loss_imputation # 填补误差和分类误差各一半
                
                # 评价指标,这个地方需要重新设计，究竟是用重构误差还是怎么着
                # （1）归一化误差
                # 填补误差，这里是重构误差
                epoch_mae_train, epoch_rmse_train, epoch_mape_train = calculate_imputation_metrics(epoch_imputed_data[epoch_train_relative_idx], epoch_target_data[epoch_train_relative_idx])
                # 验证误差，这里是重构误差，因为真值不知道，可用于中间过程早停
                epoch_mae_val, epoch_rmse_val, epoch_mape_val = calculate_imputation_metrics(epoch_imputed_data[epoch_val_relative_idx], epoch_target_data[epoch_val_relative_idx])
                # 填补误差，这里用真值，仅用于最后的结果
                epoch_mae_test, epoch_rmse_test, epoch_mape_test = calculate_imputation_metrics_mask(epoch_imputed_data[epoch_test_relative_idx], epoch_real_data[epoch_test_relative_idx], epoch_mask[epoch_test_relative_idx])
                
                # （2）反归一化误差
                # 填补误差，这里是重构误差
                epoch_de_mae_train, epoch_de_rmse_train, epoch_de_mape_train = calculate_imputation_metrics(epoch_de_imputed_data[epoch_train_relative_idx], epoch_de_target_data[epoch_train_relative_idx])
                # 验证误差，这里是重构误差，因为真值不知道，可用于中间过程早停
                epoch_de_mae_val, epoch_de_rmse_val, epoch_de_mape_val = calculate_imputation_metrics(epoch_de_imputed_data[epoch_val_relative_idx], epoch_de_target_data[epoch_val_relative_idx])
                # 填补误差，这里用真值，仅用于最后的结果
                epoch_de_mae_test, epoch_de_rmse_test, epoch_de_mape_test = calculate_imputation_metrics_mask(epoch_de_imputed_data[epoch_test_relative_idx], epoch_de_real_data[epoch_test_relative_idx], epoch_mask[epoch_test_relative_idx])
                
                # （3）分类误差
                # 分类准确率，使用epoch_estimated_label和epoch_target_labels计算
                epoch_acc_train = calculate_accuracy(epoch_estimated_labels[epoch_train_relative_idx], epoch_target_labels[epoch_train_relative_idx])
                epoch_acc_val = calculate_accuracy(epoch_estimated_labels[epoch_val_relative_idx], epoch_target_labels[epoch_val_relative_idx])
                epoch_acc_test = calculate_accuracy(epoch_estimated_labels[epoch_test_relative_idx], epoch_target_labels[epoch_test_relative_idx])

                # 记录指标
                train_losses.append(epoch_model_loss_train.item())
                val_losses.append(epoch_model_loss_val.item())
                test_losses.append(epoch_model_loss_test.item())
                train_maes.append(epoch_mae_train)
                train_rmses.append(epoch_rmse_train)
                train_mapes.append(epoch_mape_train)
                train_accs.append(epoch_acc_train)
                val_maes.append(epoch_mae_val)
                val_rmses.append(epoch_rmse_val)
                val_mapes.append(epoch_mape_val)
                val_accs.append(epoch_acc_val)
                test_maes.append(epoch_mae_test)
                test_rmses.append(epoch_rmse_test)
                test_mapes.append(epoch_mape_test)
                test_accs.append(epoch_acc_test)
                train_de_maes.append(epoch_de_mae_train)
                train_de_rmses.append(epoch_de_rmse_train)
                train_de_mapes.append(epoch_de_mape_train)
                val_de_maes.append(epoch_de_mae_val)
                val_de_rmses.append(epoch_de_rmse_val)
                val_de_mapes.append(epoch_de_mape_val)
                test_de_maes.append(epoch_de_mae_test)
                test_de_rmses.append(epoch_de_rmse_test)
                test_de_mapes.append(epoch_de_mape_test)

                # 在每个epoch的末尾计算新指标
                train_mae_acc = epoch_mae_train / epoch_acc_train
                val_mae_acc = epoch_mae_val / epoch_acc_val
                test_mae_acc = epoch_mae_test / epoch_acc_test

                # 将新指标添加到列表中
                train_mae_accs.append(train_mae_acc)
                val_mae_accs.append(val_mae_acc)
                test_mae_accs.append(test_mae_acc)

                # # 更新早停条件，使用val_mae_acc作为参考
                # if val_mae_acc < best_val_loss:  # 注意这里应根据你的指标性质决定比较方向
                #     best_val_loss = val_mae_acc
                #     triggered = 0
                # else:
                #     triggered += 1
                #     if triggered >= patience:
                #         print("Early stopping triggered at epoch {}".format(epoch))
                #         break

                # 检查早停条件
                if epoch_model_loss_val < best_val_loss:
                    best_val_loss = epoch_model_loss_val
                    triggered = 0
                else:
                    triggered += 1
                    if triggered >= patience:
                        print("Early stopping triggered at epoch {}".format(epoch))
                        break
            
            # 保存最优结果
            # 找到最小验证集损失对应的指标
            min_val_loss_index = val_losses.index(min(val_losses))

            # 构建一行数据记录
            data_record = {
                'Dataset': opt.data_name,
                'Missing Rate':opt.miss_rate,
                'Data ID':opt.id_num,
                'Model':opt.model,
                'Epoch': min_val_loss_index + 1,

                'Train Loss': train_losses[min_val_loss_index],
                'Validation Loss': val_losses[min_val_loss_index],
                'Test Loss': test_losses[min_val_loss_index],

                'Train Accuracy': train_accs[min_val_loss_index]*100,
                'Val Accuracy': val_accs[min_val_loss_index]*100,
                'Test Accuracy': test_accs[min_val_loss_index]*100,

                'Train MAE(reconstruct)': train_maes[min_val_loss_index],
                'Val MAE(reconstruct)': val_maes[min_val_loss_index],
                'Test MAE': test_maes[min_val_loss_index],

                'Train RMSE(reconstruct)': train_rmses[min_val_loss_index],
                'Val RMSE(reconstruct)': val_rmses[min_val_loss_index],
                'Test RMSE': test_rmses[min_val_loss_index],

                'Train MAPE(reconstruct)': train_mapes[min_val_loss_index],
                'Val MAPE(reconstruct)': val_mapes[min_val_loss_index],
                'Test MAPE': test_mapes[min_val_loss_index],

                'Train de_MAE(reconstruct)': train_de_maes[min_val_loss_index],
                'Val de_MAE(reconstruct)': val_de_maes[min_val_loss_index],
                'Test de_MAE': test_de_maes[min_val_loss_index],

                'Train de_RMSE(reconstruct)': train_de_rmses[min_val_loss_index],
                'Val de_RMSE(reconstruct)': val_de_rmses[min_val_loss_index],
                'Test de_RMSE': test_de_rmses[min_val_loss_index],

                'Train de_MAPE(reconstruct)': train_de_mapes[min_val_loss_index],
                'Val de_MAPE(reconstruct)': val_de_mapes[min_val_loss_index],
                'Test de_MAPE': test_de_mapes[min_val_loss_index],
                # 'Train MAE*ACC': train_mae_accs[min_val_loss_index],
                # 'Val MAE*ACC': val_mae_accs[min_val_loss_index],
                # 'Test MAE*ACC': test_mae_accs[min_val_loss_index]
            }

            #### 保存到汇总结果处
            # 总文件路径
            filename = f'zz_result/results.csv'

            # 检查文件是否存在，不存在则创建
            if not os.path.isfile(filename):
                # 创建DataFrame，并将数据记录为第一行
                df = pd.DataFrame([data_record])
            else:
                # 如果文件已存在，读取文件
                df = pd.read_csv(filename)
                # 添加新的记录
                df = df.append(data_record, ignore_index=True)

            # 保存到CSV文件
            df.to_csv(filename, index=False)

            #### 保存到按照数据集划分的文件夹，方便后续数据分析，请调用实验数据处理.ipynb
            # 分文件路径
            filename = f'zz_result/{opt.data_name}/{opt.data_name}.csv'

            # 检查文件是否存在，不存在则创建
            if not os.path.isfile(filename):
                # 创建DataFrame，并将数据记录为第一行
                df = pd.DataFrame([data_record])
            else:
                # 如果文件已存在，读取文件
                df = pd.read_csv(filename)
                # 添加新的记录
                df = df.append(data_record, ignore_index=True)

            # 保存到CSV文件
            df.to_csv(filename, index=False)

            # # 开始绘图
            # # 绘制损失曲线
            # plt.figure(figsize=(15, 9))
            # plt.subplot(3, 3, 1)
            # plt.plot(train_losses, label='Train Loss')
            # plt.plot(val_losses, label='Validation Loss')
            # plt.plot(test_losses, label='Test Loss')
            # plt.title('Loss over epochs')
            # plt.xlabel('Epoch')
            # plt.ylabel('Loss')
            # plt.legend()

            # # 绘制评价指标曲线
            # plt.subplot(3, 3, 2)
            # plt.plot(train_maes, label='Train MAE')
            # plt.plot(val_maes, label='Val MAE')
            # plt.plot(test_maes, label='Test MAE')
            # plt.title('Metrics over epochs')
            # plt.xlabel('Epoch')
            # plt.legend()

            # # 绘制评价指标曲线
            # plt.subplot(3, 3, 3)
            # plt.plot(train_de_maes, label='Train de_MAE')
            # plt.plot(val_de_maes, label='Val de_MAE')
            # plt.plot(test_de_maes, label='Test de_MAE')
            # plt.title('Metrics over epochs')
            # plt.xlabel('Epoch')
            # plt.legend()

            # # 绘制评价指标曲线
            # plt.subplot(3, 3, 4)
            # plt.plot(train_rmses, label='Train RMSE')
            # plt.plot(val_rmses, label='Val RMSE')
            # plt.plot(test_rmses, label='Test RMSE')
            # plt.title('Metrics over epochs')
            # plt.xlabel('Epoch')
            # plt.legend()

            # # 绘制评价指标曲线
            # plt.subplot(3, 3, 5)
            # plt.plot(train_de_rmses, label='Train de_RMSE')
            # plt.plot(val_de_rmses, label='Val de_RMSE')
            # plt.plot(test_de_rmses, label='Test de_RMSE')
            # plt.title('Metrics over epochs')
            # plt.xlabel('Epoch')
            # plt.legend()

            # # 绘制评价指标曲线
            # plt.subplot(3, 3, 6)
            # plt.plot(train_mapes, label='Train MAPE')
            # plt.plot(val_mapes, label='Val MAPE')
            # plt.plot(test_mapes, label='Test MAPE')
            # plt.title('Metrics over epochs')
            # plt.xlabel('Epoch')
            # plt.legend()

            # # 绘制评价指标曲线
            # plt.subplot(3, 3, 7)
            # plt.plot(train_de_mapes, label='Train de_MAPE')
            # plt.plot(val_de_mapes, label='Val de_MAPE')
            # plt.plot(test_de_mapes, label='Test de_MAPE')
            # plt.title('Metrics over epochs')
            # plt.xlabel('Epoch')
            # plt.legend()

            # # 绘制评价指标曲线
            # plt.subplot(3, 3, 8)
            # plt.plot(train_accs, label='Train Accuracy')
            # plt.plot(val_accs, label='Val Accuracy')
            # plt.plot(test_accs, label='Test Accuracy')
            # plt.title('Metrics over epochs')
            # plt.xlabel('Epoch')
            # plt.legend()

            # # 绘制MAE*ACC指标曲线
            # plt.subplot(3, 3, 9)  # 假设你已有3个图表，这是第4个
            # plt.plot(train_mae_accs, label='Train MAE/ACC')
            # plt.plot(val_mae_accs, label='Validation MAE/ACC')
            # plt.plot(test_mae_accs, label='Test MAE/ACC')
            # plt.title('MAE*ACC over epochs')
            # plt.xlabel('Epoch')
            # plt.ylabel('MAE*ACC')
            # plt.legend()

            # plt.tight_layout()
            # plt.show()

                
                
                
                
    #             print(epoch_acc_test)


    #             val_rmse, val_mae, val_mape, val_rmse_norm, val_mae_norm, val_mape_norm, val_acc = evaluate(opt, model, val_dataloader, feature_label_mask_miss, adj_euclidean, adj_mahalanobis, real_features=features_ori, norm_real_features=norm_features_ori, max_zz=max_j_zz, min_zz=min_j_zz, data_mask_add=data_mask_add)
    #             model.train()  # 确保模型处于训练模式
    #             if epoch % 1000 == 0 :
    #                 print(f"第{epoch}轮训练后，验证集的ACC为：{(val_acc*100):.2f}%,val_rmse:{val_rmse:.2f}, val_mae:{val_mae:.2f}, val_mape:{val_mape:.2f}, val_rmse_norm:{val_rmse_norm:.2f}, val_mae_norm:{val_mae_norm:.2f}, val_mape_norm:{val_mape_norm:.2f}")
    # # ######################### 新版本早停  #############################
    #             # 更新ACC早停逻辑
    #             if val_acc > best_acc:
    #                 best_acc = val_acc
    #                 wait_acc = 0
    #                 # 保存当前最优模型状态 - 基于ACC
    #                 torch.save(model.state_dict(), opt.model_save_path_acc)
    #             else:
    #                 wait_acc += 1
    #                 if wait_acc >= patience_acc and not test_performed_acc:
    #                     early_stop_triggered_acc = True
    #                     print(f'模型按ACC标准早停, 运行轮数为{epoch}, 最好的验证集分类精度为: {best_acc*100:.2f}%')
    #                     # 执行基于ACC的测试并记录结果
    #                     test_performed_acc = True

    #             # 更新MAE早停逻辑
    #             if val_mae < best_mae:
    #                 best_mae = val_mae 
    #                 wait_mae = 0
    #                 # 保存当前最优模型状态 - 基于MAE
    #                 torch.save(model.state_dict(), opt.model_save_path_mae)
    #             else:
    #                 wait_mae += 1
    #                 if wait_mae >= patience_mae and not test_performed_mae:
    #                     early_stop_triggered_mae = True
    #                     print(f'模型按MAE标准早停, 运行轮数为{epoch}, 最低的验证集MAE为: {best_mae:.2f}')
    #                     # 执行基于MAE的测试并记录结果
    #                     test_performed_mae = True

    #             # 检查是否已经触发了基于两个指标的早停并执行了测试
    #             if test_performed_acc and test_performed_mae:
    #                 print("基于ACC和MAE的早停条件都已满足，训练结束。")
    #                 print("#####"*10)
    #                 break

    #         # 测试阶段，调用保存的模型状态进行

    #         # 根据ACC的最优模型状态进行测试
    #         print("加载基于ACC的最优模型状态进行测试...")
    #         model.load_state_dict(torch.load(opt.model_save_path_acc))
    #         # 在测试集上评估模型，寻找最佳性能
    #         rmse, mae, mape, rmse_norm, mae_norm, mape_norm, acc = evaluate(opt, model, test_dataloader, feature_label_mask_miss, adj_euclidean, adj_mahalanobis, real_features=features_ori, norm_real_features=norm_features_ori, max_zz=max_j_zz, min_zz=min_j_zz, data_mask_add=data_mask_add)
    #         print(f"在测试集上的mae是{mae:.2f}, mape是{mape:.2f}, acc是{acc*100:.0f}%, norm_mae是{mae_norm:.2f}")
    #         # 记录测试结果
    #         writeline_csv("zz_result/record_acc.csv", rmse, mae, mape, acc, rmse_norm, mae_norm, mape_norm)

    #         # 根据MAE的最优模型状态进行测试
    #         print("加载基于MAE的最优模型状态进行测试...")
    #         model.load_state_dict(torch.load(opt.model_save_path_mae))
    #         # 在测试集上评估模型，寻找最佳性能
    #         rmse, mae, mape, rmse_norm, mae_norm, mape_norm, acc = evaluate(opt, model, test_dataloader, feature_label_mask_miss, adj_euclidean, adj_mahalanobis, real_features=features_ori, norm_real_features=norm_features_ori, max_zz=max_j_zz, min_zz=min_j_zz, data_mask_add=data_mask_add)
    #         print(f"在测试集上的mae是{mae:.2f}, mape是{mape:.2f}, acc是{acc*100:.0f}%, norm_mae是{mae_norm:.2f}")
    #         print("-----"*10)
    #         # 记录测试结果
    #         writeline_csv("zz_result/record_mae.csv", rmse, mae, mape, acc, rmse_norm, mae_norm, mape_norm)
    # # ######################### 旧版本早停  #############################甚至没有保存模型状态
    # #             # 检查是否需要早停
    # #             if val_acc > best_acc:
    # #                 best_acc = val_acc
    # #                 wait = 0  # 重置等待计数器
    # #             else:
    # #                 wait += 1
    # #                 if wait >= patience:
    # #                     print(f'模型早停, 运行轮数为{epoch}, 最好的验证集分类精度为: {best_acc*100:.2f}%')
    # #                     break  # 早停
    # #         # 在测试集上评估模型，寻找最佳性能
    # #         rmse, mae, mape, rmse_norm, mae_norm, mape_norm, acc = evaluate(opt, model, test_dataloader, feature_label_mask_miss, adj_euclidean, adj_mahalanobis, real_features=features_ori, norm_real_features=norm_features_ori, max_zz=max_j_zz, min_zz=min_j_zz, data_mask_add=data_mask_add)
    # #         print(f"在测试集上，最终的mae是{mae:.2f}, mape是{mape:.2f}, acc是{acc*100:.0f}%, norm_mae是{mae_norm:.2f}")
    # #         print("-----"*10)
    # #         # 记录测试结果
    # #         writeline_csv("zz_result/record.csv", rmse, mae, mape, acc, rmse_norm, mae_norm, mape_norm)
